## Introduction to Kernel Methods

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

We will look at some first examples with synthetic data. We'll compare how different classical and kernel methods perform when trying to find the best label predictions for synthetic data points. The synthetic data points $x_1, x_2, \ldots, x_n$ are obtained by sampling random values from the uniform distribution. Their labels $y_1, y_2, \ldots, y_n$ are generated using a non-linear target function defined as:

$ y_i = f(x_i) = 10 \left( \sin(3(1 - x_i)) \cdot \tanh(5(1 - x_i)) + \left(\frac{x_i - 0.3}{0.9}\right)^2 - 0.514 \right) $

In the code, we have $X = x_1, x_2, \ldots, x_n$ and $y = y_1, y_2, \ldots, y_n$.

In [ ]:
def nonlinear_target_function(x):
    """
    param `x` (data points for which the target y is generated)
    """
    return 10 * (np.sin(3 * (1 - x)) * np.tanh(5 * (1 - x)) + ((x-0.3)/0.9)**2 - 0.514)

def generate_data(n, sigma=2, low = -1, high = 1):
    """
    param `n`: number of data points to generate
    param `sigma`: standard deviation of noise for the uniform distribution
    param `low`: lower bound of data points
    param `high`: upper bound of data points
    """
    rng = np.random.default_rng()
    X = rng.uniform(low=low, high=high, size=(n,))
    
    targets = nonlinear_target_function(X)
    y = targets + sigma * rng.normal(size=(n,))
    
    return X, y

# Here we generate 100 data points X and their corresponding labels y
n = 100
X, y = generate_data(n)

Here we visualize both the target function and the generated data. The target function is plotted using a continuous line, while the data points are scattered in red. The **target function** illustrates the true pattern that we hope to find using different methods, and the **data points** show the noisy observations.

In [ ]:
low, high = -1, 1
xs = np.linspace(low, high, 100)
plt.plot(xs, nonlinear_target_function(xs), label="Target function")
plt.scatter(X, y, label="Data", marker='.', color="red")
plt.legend()
plt.show()

The *eval_solution* function evaluates the performance of a predictor function and visualizes its predictions compared to the true target function.

In [ ]:
def eval_solution(predictor, X, y):
    """
    param `predictor`: a function that takes a single input and returns a prediction
    param `X`: input data points used for training
    param `y`: true labels corresponding to the training data
    """
    predictions_train = np.array([predictor(x) for x in X])
    mse_train = np.mean((y - predictions_train)**2)
    print(f"MSE on training data : {mse_train:.2f}")
    
    xs = np.linspace(low, high, 500)
    predictions = np.array([predictor(x) for x in xs])
    targets = nonlinear_target_function(xs)
    mse = np.mean((targets - predictions)**2)
    print(f"MSE to target function: {mse:.2f}")

    plt.plot(xs, targets, label="Target")
    plt.scatter(X, y, label="Data", marker='.', color="red")
    plt.plot(xs, predictions, label="Predictions")
    plt.legend()
    plt.show()

### Linear Regression

Here we use a classical linear model: *Linear Regression* to find a linear prediction function for the data.

In [ ]:
from sklearn.linear_model import LinearRegression

# Fit a linear regression model
##### Fill here ! #####

# Obtain the coefficients
alpha = ##### Fill here ! #####

# Create the linear predictor function
predictor = lambda x: alpha * x

# Evaluate the solution
eval_solution(predictor, X, y)

### Kernel Regression on Vectors

In machine learning, a kernel is a function that computes the similarity (or inner product) between two data points in a higher-dimensional space. Mathematically, a kernel $k$ between two data points $x$ and $x'$ is defined as $k(x, x') = \langle\psi(x), \psi(x')\rangle_X$, where $\psi$ is a function that maps the data points into a higher-dimensional space. Here we try to find different prediction functions for our synthetic data using different kernels such as the linear kernel, the polynomial kernel, and the Gaussian (RBF) kernel.

In [ ]:
import numpy as np

def compute_kernel_matrix(kernel, X):
    """
    Compute the kernel matrix for a given kernel function and input data.

    Parameters:
    - kernel: The kernel function.
    - X: Input data.

    Returns:
    - K: The computed kernel matrix.
    """
    n = len(X)
    K = np.empty((n, n))
    for i in range(n):
        for j in range(n):
            # Calculate the kernel value for each pair of data points.
            ##### Fill here ! #####
    return K

def kernel_ridge(K, y, lbd=0.1):
    """
    Perform kernel ridge regression.

    Parameters:
    - K: The kernel matrix.
    - y: Target values.
    - lbd: Regularization parameter.

    Returns:
    - alpha: Coefficients obtained from kernel ridge regression.
    """
    n = len(K)
    # Solve the linear system using the kernel matrix, target values, and regularization parameter.
    alpha, _, _, _ = np.linalg.lstsq(K + lbd * np.identity(n), y, rcond=None)
    return alpha

def make_predictor(kernel, X, alpha):
    """
    Create a predictor function based on the kernel, input data, and obtained coefficients.

    Parameters:
    - kernel: The kernel function.
    - X: Input data.
    - alpha: Coefficients obtained from kernel ridge regression.

    Returns:
    - predictor: The predictor function.
    """
    n = len(X)
    def predictor(x):
        # Use the obtained coefficients to make predictions for a new data point.
        prediction = 0
        for i in range(n):
            ##### Fill here ! #####
        return prediction
    return predictor


In [ ]:
# Define a linear kernel function
def linear_kernel(x, xp):
    """
    Linear kernel function.

    Parameters:
    - x: First data point.
    - xp: Second data point.

    Returns:
    - Result of the linear kernel between x and xp.
    """
    return ##### Fill here ! #####

# Compute the kernel matrix using the linear kernel
K = compute_kernel_matrix(linear_kernel, X)

# Perform kernel ridge regression to obtain coefficients
alpha = kernel_ridge(K, y)

# Create a predictor function based on the linear kernel and obtained coefficients
predictor = make_predictor(linear_kernel, X, alpha)

# Evaluate the performance of the linear predictor
eval_solution(predictor, X, y)

In [ ]:
# Define a second-degree polynomial kernel function
def polynomial_kernel_2(x, xp):
    """
    Second-degree polynomial kernel function.

    Parameters:
    - x: First data point.
    - xp: Second data point.

    Returns:
    - Result of the polynomial kernel of degree 2 between x and xp.
    """
    return ##### Fill here ! (look at the expression for the 3d order pol. kernel below) #####

# Compute the kernel matrix using the second-degree polynomial kernel
K = compute_kernel_matrix(polynomial_kernel_2, X)

# Perform kernel ridge regression to obtain coefficients
alpha = kernel_ridge(K, y)

# Create a predictor function based on the second-degree polynomial kernel and obtained coefficients
predictor = make_predictor(polynomial_kernel_2, X, alpha)

# Evaluate the performance of the polynomial predictor
eval_solution(predictor, X, y)

In [ ]:
# Define a third-degree polynomial kernel function
def polynomial_kernel_3(x, xp):
    """
    Third-degree polynomial kernel function.

    Parameters:
    - x: First data point.
    - xp: Second data point.

    Returns:
    - Result of the polynomial kernel of degree 3 between x and xp.
    """
    return x * xp + x**2 * xp**2 + x**3 * xp**3

# Compute the kernel matrix using the third-degree polynomial kernel
K = compute_kernel_matrix(polynomial_kernel_3, X)

# Perform kernel ridge regression to obtain coefficients
alpha = kernel_ridge(K, y)

# Create a predictor function based on the third-degree polynomial kernel and obtained coefficients
predictor = make_predictor(polynomial_kernel_3, X, alpha)

# Evaluate the performance of the polynomial predictor
eval_solution(predictor, X, y)

In [ ]:
# Set the bandwidth for the Gaussian kernel
bw = 0.5

# Define a Gaussian kernel function
def gaussian_kernel(x, xp):
    """
    Gaussian kernel function.

    Parameters:
    - x: First data point.
    - xp: Second data point.

    Returns:
    - Result of the Gaussian kernel between x and xp.
    """
    return np.exp(-(x - xp)**2 / (2 * bw**2))

# Compute the kernel matrix using the Gaussian kernel
K = compute_kernel_matrix(gaussian_kernel, X)

# Perform kernel ridge regression to obtain coefficients
alpha = kernel_ridge(K, y)

# Create a predictor function based on the Gaussian kernel and obtained coefficients
predictor = make_predictor(gaussian_kernel, X, alpha)

# Evaluate the performance of the Gaussian predictor
eval_solution(predictor, X, y)


A Gaussian kernel with a too small bandwidth *bw* can lead to overfitting in kernel methods. Overfitting occurs when a model learns to perform well on the training data but fails to generalize to new, unseen data. A small bandwidth in the Gaussian kernel results in a narrow Gaussian function. This means that the kernel assigns significant weight only to points very close to each data point. As a result, the model becomes highly sensitive to the specificities of the training data, capturing noise and outliers rather than the true underlying pattern (target function).

In [ ]:
# Set the bandwidth for the Gaussian kernel
bw = 0.01

# Compute the kernel matrix using the Gaussian kernel
K = compute_kernel_matrix(gaussian_kernel, X)

# Perform kernel ridge regression to obtain coefficients
alpha = kernel_ridge(K, y)

# Create a predictor function based on the Gaussian kernel and obtained coefficients
predictor = make_predictor(gaussian_kernel, X, alpha)

# Evaluate the performance of the Gaussian predictor
eval_solution(predictor, X, y)


In [ ]:
# Define a minimum kernel function
def min_kernel(x, xp):
    """
    Minimum kernel function.

    Parameters:
    - x: First data point.
    - xp: Second data point.

    Returns:
    - Minimum value between x and xp.
    """
    return ##### Fill here ! #####

# Compute the kernel matrix using the minimum kernel
K = compute_kernel_matrix(min_kernel, X)

# Perform kernel ridge regression to obtain coefficients
alpha = kernel_ridge(K, y)

# Create a predictor function based on the minimum kernel and obtained coefficients
predictor = make_predictor(min_kernel, X, alpha)

# Evaluate the performance of the minimum kernel predictor
eval_solution(predictor, X, y)

The sum kernel is formed by summing the minimum and Gaussian kernels as follows:

$K_{\text{sum}}(x, xp) = K_{\text{min}}(x, xp) + K_{\text{RBF}}(x, xp)$

The sum kernel incorporates both the local agreement captured by the minimum kernel and the smooth, global relationships captured by the Gaussian kernel. This combination allows the sum kernel to balance the importance of local and global structures in the data.

In [ ]:
# Set the bandwidth for the Gaussian kernel
bw = 0.5

# Define a sum kernel by combining the minimum and Gaussian kernels
def sum_kernel(x, xp):
    """
    Sum kernel function.

    Parameters:
    - x: First data point.
    - xp: Second data point.

    Returns:
    - Sum of the minimum and Gaussian kernels between x and xp.
    """
    return ##### Fill here ! #####

# Compute the kernel matrix using the sum kernel
K = compute_kernel_matrix(sum_kernel, X)

# Perform kernel ridge regression to obtain coefficients
alpha = kernel_ridge(K, y)

# Create a predictor function based on the sum kernel and obtained coefficients
predictor = make_predictor(sum_kernel, X, alpha)

# Evaluate the performance of the sum kernel predictor
eval_solution(predictor, X, y)

## Kernel Regression on Graphs

In [ ]:
import networkx as nx
from gklearn.utils.graph_files import load_dataset
%matplotlib inline

When applied to graphs, kernel methods allow us to exploit the structural information encoded in graph data. In this context, each graph is treated as a data point, and kernels measure the similarity between pairs of graphs.

*The following code is adapted from a practical by https://scholar.google.com/citations?user=YqmqE9gAAAAJ&hl=en. The dataset a modified version of https://brunl01.users.greyc.fr/CHEMISTRY/index.html#Acyclic.*

We first load a graph dataset (`acyclic/dataset_bps.ds`) along with associated target values. The dataset consists of graphs (molecules) and corresponding boiling temperature values.

In [ ]:
G, y, info = load_dataset("acyclic/dataset_bps.ds")
y = np.array(y)
N = len(G)

Some exploratory analysis is performed on the dataset, such as visualizing two example graphs, computing the mean and variance of the target temperatures.

In [ ]:
# Visualizing Two Graphs Side by Side

# Plotting the first graph (G1)
x1 = 92
G1 = G[x1]
print("Label for G1", y[x1])
label = "atom_symbol"
print("Atoms of G1", nx.get_node_attributes(G1 ,label))
plt.subplot(1, 2, 1)  # Subplot for the first graph
nx.draw_networkx(G1)
plt.title(f'Graph {x1} (Label: {y[x1]:.2f})')

# Plotting the second graph (G2)
x2 = 90
G2 = G[x2]
print("Label for G2", y[x2])
print("Atoms of G2", nx.get_node_attributes(G2 ,label))
plt.subplot(1, 2, 2)  # Subplot for the second graph
nx.draw_networkx(G2)
plt.title(f'Graph {x2} (Label: {y[x2]:.2f})')

plt.show()

# Displaying Mean and Variance of Temperatures
print("Mean temperature", np.mean(y))
print("Variance of temperature", np.mean((y - np.mean(y))**2))

In [ ]:
from sklearn.svm import SVR
import numpy as np
from sklearn.model_selection import ShuffleSplit

def train_and_evaluate_graph_kernel_method(K):

    # Visualize the computed kernel matrix as an image plot
    plt.imshow(K)

    # Compute and print the minimum eigenvalue of the kernel matrix
    print("Min eigenvalue: ", np.min(np.linalg.eigvalsh(K)))

    # Initialize Support Vector Regression (SVR) with a precomputed kernel
    clf = SVR(kernel="precomputed")

    # Define a random splitting of the dataset
    rs = ShuffleSplit(n_splits=5, test_size=0.33, random_state=0)

    # Initialize an empty list to store errors
    errors = []

    # Create an array representing the dataset indices
    dataset = np.arange(len(G))

    # Iterate through train and validation sets in each split
    for train_index, val_index in rs.split(dataset):
        # Extract training and validation kernel matrices
        Ktrain = K[train_index, :][:, train_index]
        Kval = K[val_index, :][:, train_index]
        
        # Fit SVR model on training data
        clf.fit(Ktrain, y[train_index])
        
        # Predict on the validation set
        y_pred = clf.predict(Kval)
        
        # Calculate and store absolute errors
        errors.extend(np.abs(y_pred - y[val_index]))

    # Print the Mean Squared Error (MSE) over all splits
    print("MSE:", np.mean(errors))

To first apply kernel methods for vectors, we convert the categorical node attribute "atom_symbol" into a numerical format using a simple feature map (fingerprint). This transformation results in a binary vector representation for each graph, capturing the presence or absence of different atom symbols.

In [ ]:
def fingerprint(G):
    """
    Generate a fingerprint matrix for a list of graphs based on node attributes.

    Parameters:
    - G: List of graphs.

    Returns:
    - Fingerprint matrix with one-hot encodings for node attributes.
    """

    # Get unique atom symbols from all graphs and sort them
    unique_labels = list(set(l for Gi in G for l in nx.get_node_attributes(Gi, "atom_symbol").values()))
    unique_labels.sort()

    # Create an empty numpy array to store the fingerprint encodings
    num_dicts = len(G)
    num_labels = len(unique_labels)
    encoded_array = np.zeros((num_dicts, num_labels), dtype=int)

    # Map labels to their index in unique_labels
    label_to_index = {label: i for i, label in enumerate(unique_labels)}

    # Populate the numpy array with encodings for each dictionary
    for i, Gi in enumerate(G):
        for key, label in nx.get_node_attributes(Gi, "atom_symbol").items():
            index = label_to_index[label]
            encoded_array[i, index] = 1

    return encoded_array

We use a radial basis function (RBF) kernel to measure the similarity between graphs. The RBF kernel captures the structural similarity between graphs based on fingerprint representations.

In [ ]:
# Generate fingerprint features for all graphs
features = fingerprint(G)
print(f'Graph {x1} (Fingerprint: {features[x1]})')
print(f'Graph {x2} (Fingerprint: {features[x2]})')

Now we use the Support Vector Regression (SVR) model with an RBF kernel on the graph fingerprints to predict target values. SVR is a machine learning algorithm that finds a hyperplane in a high-dimensional space to minimize the regression error. The kernel matrix computed from the RBF kernel is used as input to SVR, allowing the algorithm to operate directly on the graph similarities.

In [ ]:
def rbf_kernel(x, y, sigma=1):
    """
    Radial Basis Function (RBF) kernel function.

    Parameters:
    - x: First data point.
    - y: Second data point.
    - sigma: RBF kernel parameter (default is 1).

    Returns:
    - Result of the RBF kernel between x and y.
    """
    return np.exp(- (np.linalg.norm(x - y) ** 2) / (2 * sigma ** 2))

# Initialize an empty kernel matrix
n = len(features)
K_fp = np.zeros((n, n))

# Compute the kernel matrix using the RBF kernel
for i in range(n):
    for j in range(n):
        K_fp[i, j] = rbf_kernel(features[i], features[j])

train_and_evaluate_graph_kernel_method(K_fp)

Now we use the treelet kernel, a krenel that works **directly on the graphs** rather than their vector representations. Treelets are graph fragments that capture local substructures. The treelet kernel measures the similarity between graphs based on the number of common treelets, providing a more interpretable notion of similarity.

In [ ]:
from gklearn.kernels import treeletKernel

# Compute the treelet kernel matrix for the entire dataset of graphs
K_tree, run_time = treeletKernel.treeletkernel(##### Fill here ! #####)

train_and_evaluate_graph_kernel_method(K_tree)

Since the Treelet kernel does not perform better than the regressing on fingerprints, we use another kernel that works **directly on the graphs**. The Weisfeiler-Lehman (WL) kernel was introduced by Benjamin Weisfeiler and Alexander Lehman in 1968, it works by running a labelling-aggregation algorithm on both graphs and comparing the two algorithm runs.

In [ ]:
from gklearn.kernels import weisfeilerLehmanKernel

# Compute the Weisfeiler Lehman kernel matrix for the entire dataset of graphs
K_WL, run_time = weisfeilerLehmanKernel.weisfeilerlehmankernel(##### Fill here ! #####)

train_and_evaluate_graph_kernel_method(K_WL)


In [ ]:
# Pair-wise sum of positive definite kernels is a positive definite kernel
K_sum = ##### Fill here ! #####

train_and_evaluate_graph_kernel_method(K_sum)

In [ ]:
# Pair-wise product of positive definite kernels is a positive definite kernel
K_prod = ##### Fill here ! #####

train_and_evaluate_graph_kernel_method(K_prod)